Model Without Pruning

In [ ]:
import tensorflow as tf
import tensorflow_model_optimization as tfmot
import numpy as np

# Load and preprocess MNIST dataset
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()
train_images = train_images.reshape((60000, 28, 28, 1)).astype('float32') / 255
test_images = test_images.reshape((10000, 28, 28, 1)).astype('float32') / 255

# One-hot encode the labels
train_labels = tf.keras.utils.to_categorical(train_labels, 10)
test_labels = tf.keras.utils.to_categorical(test_labels, 10)

# Manually recreate the model architecture
def create_model():
    model = tf.keras.Sequential([
        tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)),
        tf.keras.layers.MaxPooling2D((2, 2)),
        tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
        tf.keras.layers.MaxPooling2D((2, 2)),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(10, activation='softmax')
    ])
    return model

# Create and load weights manually
model = create_model()
loaded_model = tf.keras.models.load_model('mnist_trained_model1.h5')

# Copy weights from loaded model to our manually created model
for i, layer in enumerate(loaded_model.layers):
    model.layers[i].set_weights(layer.get_weights())

# Compile the model
model.compile(optimizer='adam', 
              loss='categorical_crossentropy', 
              metrics=['accuracy'])

# Print initial model information
print("Original Model Summary:")
model.summary()
model.fit(
    train_images, train_labels, 
    epochs=5, 
    batch_size=64, 
    validation_data=(test_images, test_labels)
)

# Evaluate the model
test_loss_model, test_accuracy_model = model.evaluate(test_images, test_labels)
print(f"\nTest accuracy : {test_accuracy_model:.4f}")


: 

Model after Prune

In [13]:
import tensorflow as tf
import tensorflow_model_optimization as tfmot
import numpy as np

# Load and preprocess MNIST dataset
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()
train_images = train_images.reshape((60000, 28, 28, 1)).astype('float32') / 255
test_images = test_images.reshape((10000, 28, 28, 1)).astype('float32') / 255

# One-hot encode the labels
train_labels = tf.keras.utils.to_categorical(train_labels, 10)
test_labels = tf.keras.utils.to_categorical(test_labels, 10)

# Manually recreate the model architecture
def create_model():
    model = tf.keras.Sequential([
        tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)),
        tf.keras.layers.MaxPooling2D((2, 2)),
        tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
        tf.keras.layers.MaxPooling2D((2, 2)),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(10, activation='softmax')
    ])
    return model

# Create and load weights manually
model = create_model()
loaded_model = tf.keras.models.load_model('mnist_trained_model1.h5')

# Copy weights from loaded model to our manually created model
for i, layer in enumerate(loaded_model.layers):
    model.layers[i].set_weights(layer.get_weights())

# Compile the model
model.compile(optimizer='adam', 
              loss='categorical_crossentropy', 
              metrics=['accuracy'])

# Manual pruning approach
def manual_prune_weights(weights, pruning_threshold=0.5):
    """
    Manually prune weights by setting low-magnitude weights to zero
    
    Args:
    weights (numpy.ndarray): Weight matrix to prune
    pruning_threshold (float): Threshold for pruning, based on absolute weight magnitude
    
    Returns:
    numpy.ndarray: Pruned weight matrix
    """
    # Calculate the threshold based on weight magnitudes
    threshold = np.percentile(np.abs(weights), pruning_threshold * 100)
    
    # Set weights below threshold to zero
    pruned_weights = weights.copy()
    pruned_weights[np.abs(pruned_weights) < threshold] = 0
    
    return pruned_weights

# Apply pruning to the model
pruned_layers = []
for layer in model.layers:
    if isinstance(layer, (tf.keras.layers.Dense, tf.keras.layers.Conv2D)):
        # Get current weights
        weights = layer.get_weights()
        
        # Prune weights manually
        pruned_weights = []
        for weight_matrix in weights:
            pruned_weights.append(manual_prune_weights(weight_matrix))
        
        # Set pruned weights
        layer.set_weights(pruned_weights)
    
    pruned_layers.append(layer)

# Recreate the model with pruned layers
pruned_model = tf.keras.Sequential(pruned_layers)

# Compile the pruned model
pruned_model.compile(optimizer='adam', 
                     loss='categorical_crossentropy', 
                     metrics=['accuracy'])

# Train the pruned model
pruned_model.fit(
    train_images, train_labels, 
    epochs=5, 
    batch_size=64, 
    validation_data=(test_images, test_labels)
)

# Evaluate the pruned model
test_loss, test_accuracy = pruned_model.evaluate(test_images, test_labels)
print(f"\nTest accuracy after pruning: {test_accuracy:.4f}")

# Calculate model sparsity
def calculate_model_sparsity(model):
    total_weights = 0
    zero_weights = 0
    for layer in model.layers:
        for weight in layer.get_weights():
            total_weights += np.prod(weight.shape)
            zero_weights += np.sum(weight == 0)
    
    return zero_weights / total_weights

sparsity = calculate_model_sparsity(pruned_model)
print(f"\nModel Sparsity: {sparsity:.4f}")

# Save the pruned model
pruned_model.save('mnist_pruned_model.h5')
pruned_model.save('mnist_pruned_model.keras')

C:\Users\ASUS\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/5
938/938 ━━━━━━━━━━━━━━━━━━━━ 13s 13ms/step - accuracy: 0.9954 - loss: 0.0145 - val_accuracy: 0.9922 - val_loss: 0.0269
Epoch 2/5
938/938 ━━━━━━━━━━━━━━━━━━━━ 12s 13ms/step - accuracy: 0.9982 - loss: 0.0062 - val_accuracy: 0.9912 - val_loss: 0.0287
Epoch 3/5
938/938 ━━━━━━━━━━━━━━━━━━━━ 15s 15ms/step - accuracy: 0.9978 - loss: 0.0070 - val_accuracy: 0.9904 - val_loss: 0.0348
Epoch 4/5
938/938 ━━━━━━━━━━━━━━━━━━━━ 13s 14ms/step - accuracy: 0.9981 - loss: 0.0061 - val_accuracy: 0.9911 - val_loss: 0.0333
Epoch 5/5
938/938 ━━━━━━━━━━━━━━━━━━━━ 13s 14ms/step - accuracy: 0.9988 - loss: 0.0036 - val_accuracy: 0.9884 - val_loss: 0.0441
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9853 - loss: 0.0576



Test accuracy after pruning: 0.9884

Model Sparsity: 0.1618


Quantization

In [ ]:
import tensorflow as tf
from tensorflow import keras

# Load the Keras model
mnist_pruned_model = keras.models.load_model('mnist_pruned_model.h5')

# Convert the trained model to TensorFlow Lite format
converter = tf.lite.TFLiteConverter.from_keras_model(mnist_pruned_model)

# Enable optimizations for quantization
converter.optimizations = [tf.lite.Optimize.DEFAULT]

# Convert the model
quantized_model = converter.convert()

# Save the quantized model
with open("mnist_model_quantized.h5", "wb") as f:
    f.write(quantized_model)


INFO:tensorflow:Assets written to: C:\Users\ASUS\AppData\Local\Temp\tmprj8ijhl7\assets


INFO:tensorflow:Assets written to: C:\Users\ASUS\AppData\Local\Temp\tmprj8ijhl7\assets


Saved artifact at 'C:\Users\ASUS\AppData\Local\Temp\tmprj8ijhl7'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 28, 28, 1), dtype=tf.float32, name='input_layer_6')
Output Type:
  TensorSpec(shape=(None, 10), dtype=tf.float32, name=None)
Captures:
  1776207075792: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1776207069648: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1776207073104: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1776260925200: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1776260925008: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1776260927312: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1776260925968: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1776260927696: TensorSpec(shape=(), dtype=tf.resource, name=None)
